In [ ]:
import pandas as pd
import requests
import csv
import time

def extract_restaurant_and_market_items(city_data):
    items = []
    for item in city_data.get("prices", []):
        if item.get("category_name") in ("Restaurants", "Markets"):
            items.append({
                "item_name": item.get("item_name"),
                "category": item.get("category_name"),
                "min": item.get("min"),
                "avg": item.get("avg"),
                "max": item.get("max"),
                "currency": item.get("currency_code"),
                "usd_min": item.get("usd", {}).get("min"),
                "usd_avg": item.get("usd", {}).get("avg"),
                "usd_max": item.get("usd", {}).get("max"),
            })
    return items
    

In [ ]:
API_URL = "https://cost-of-living-and-prices.p.rapidapi.com/prices"

API_KEYS = [
    "8ac6d68495msh8602e0bc8f6b94dp1531adjsnf1227f873f48",
    "4a8b5079bfmsh08e3b5ef702cc66p18aeabjsncfb92b18764b",
    "72af03b191mshe053d673dd4b88dp197dedjsn5be31fb1513d",
    "7fdb72c360msh02214090800cd2bp1892f8jsn1a840290b2a1"
]

API_HOST = "cost-of-living-and-prices.p.rapidapi.com"

CITIES = [
    ("London", "United Kingdom"),
    ("New York", "United States"),
    ("Los Angeles", "United States"),
    ("Chicago", "United States"),
    ("Toronto", "Canada"),
    ("Vancouver", "Canada"),
    ("Rome", "Italy"),
    ("Barcelona", "Spain"),
    ("Amsterdam", "Netherlands"),
    ("Prague", "Czech Republic"),
    ("Zurich", "Switzerland"),
    ("Geneva", "Switzerland"),
    ("Copenhagen", "Denmark"),
    ("Oslo", "Norway"),
    ("Gothenburg", "Sweden"),
    ("Stockholm", "Sweden"),
    ("Vienna", "Austria"),
    ("Berlin", "Germany"),
    ("Munich", "Germany"),
    ("Frankfurt", "Germany"),
    ("Hamburg", "Germany"),
    ("Cologne", "Germany"),
    ("Dusseldorf", "Germany"),
    ("Stuttgart", "Germany"),
    ("Helsinki", "Finland"),
    ("Paris", "France"),
    ("Lyon", "France"),
    ("Toulouse", "France"),
    ("Madrid", "Spain"),
    ("Valencia", "Spain"),
    ("Seville", "Spain"),
    ("Lisbon", "Portugal"),
    ("Porto", "Portugal"),
    ("Braga", "Portugal"),
    ("Coimbra", "Portugal"),
    ("Athens", "Greece"),
    ("Thessaloniki", "Greece"),
    ("Warsaw", "Poland"),
    ("Krakow", "Poland"),
    ("Wroclaw", "Poland"),
    ("Poznan", "Poland"),
    ("Gdansk", "Poland"),
    ("Lodz", "Poland"),
    ("Szczecin", "Poland"),
    ("Katowice", "Poland"),
    ("Lublin", "Poland"),
    ("Budapest", "Hungary"),
    ("Debrecen", "Hungary"),
    ("Pecs", "Hungary"),
    ("Brno", "Czech Republic"),
    ("Ostrava", "Czech Republic"),
    ("Plzen", "Czech Republic"),
    ("Olomouc", "Czech Republic"),
    ("Brussels", "Belgium"),
    ("Antwerp", "Belgium"),
    ("Rotterdam", "Netherlands"),
    ("The Hague", "Netherlands"),
    ("Utrecht", "Netherlands"),
    ("Eindhoven", "Netherlands"),
    ("Maastricht", "Netherlands"),
    ("Groningen", "Netherlands"),
    ("Nijmegen", "Netherlands"),
    ("Singapore", "Singapore"),
    ("Dubai", "United Arab Emirates"),
    ("Hong Kong", "Hong Kong (China)"),
    ("Seoul", "South Korea"),
    ("Tokyo", "Japan"),
    ("Shanghai", "China"),
    ("Beijing", "China"),
    ("Guangzhou", "China"),
    ("Shenzhen", "China"),
    ("Kuala Lumpur", "Malaysia"),
    ("Bangkok", "Thailand"),
    ("Jakarta", "Indonesia"),
    ("Istanbul", "Turkey"),
    ("Melbourne", "Australia"),
    ("Brisbane", "Australia"),
    ("Sydney", "Australia"),
    ("Perth", "Australia"),
    ("Canberra", "Australia"),
    ("Adelaide", "Australia"),
    ("Auckland", "New Zealand"),
    ("Wellington", "New Zealand"),
    ("Christchurch", "New Zealand"),
    ("San Francisco", "United States"),
    ("Seattle", "United States"),
    ("Washington", "United States"),
    ("Boston", "United States"),
    ("San Jose", "United States"),
    ("San Diego", "United States"),
    ("Atlanta", "United States"),
    ("Denver", "United States"),
    ("Austin", "United States"),
    ("Dallas", "United States"),
    ("Houston", "United States"),
    ("Philadelphia", "United States"),
    ("Miami", "United States"),
    ("Las Vegas", "United States"),
    ("Phoenix", "United States"),
    ("Ottawa", "Canada"),
    ("Montreal", "Canada"),
    ("Calgary", "Canada"),
    ("Edmonton", "Canada"),
    ("Winnipeg", "Canada"),
    ("Santiago", "Chile"),
    ("Sao Paulo", "Brazil"),
    ("Buenos Aires", "Argentina"),
    ("Lima", "Peru"),
    ("Bogota", "Colombia"),
    ("Rio de Janeiro", "Brazil")
]

results = []

REQUESTS_PER_KEY = 10
TOTAL_REQUESTS_PER_HOUR = REQUESTS_PER_KEY * len(API_KEYS)

request_count = 0

for i, (city, country) in enumerate(CITIES, start=1):

    # Rotate API key
    key_index = request_count % len(API_KEYS)
    headers = {
        "x-rapidapi-key": API_KEYS[key_index],                                                                         
        "x-rapidapi-host": API_HOST
    }

    params = {
        "city_name": city,
        "country_name": country
    }

    print(f"Request {i}: {city}, {country} using key {key_index + 1}")

    response = requests.get(API_URL, headers=headers, params=params)
    data = response.json()

    items = extract_restaurant_and_market_items(data)

    for it in items:
        results.append({
            "city": city,
            "country": country,
            **it
        })

    request_count += 1
    
    if request_count % TOTAL_REQUESTS_PER_HOUR == 0 and i < len(CITIES):
        print("⏳ 40 requests reached. Sleeping for 1 hour...")
        time.sleep(3600)



# Convert to DataFrame
df = pd.DataFrame(results)
print(df)

# Save CSV
csv_file = "prices.csv"

with open(csv_file, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow([
        "City", "Country", "Category", "Item Name",
        "Min Local", "Avg Local", "Max Local",
        "Currency", "Min USD", "Avg USD", "Max USD"
    ])

    for row in results:
        writer.writerow([
            row["city"], row["country"], row["category"], row["item_name"],
            row["min"], row["avg"], row["max"],
            row["currency"], row["usd_min"], row["usd_avg"], row["usd_max"]
        ])

print("DONE! Saved to prices.csv")